In [0]:
import yaml
spark

In [0]:
with open("/Workspace/Repos/ziaarzoo21@gmail.com/spark-etl/config/config.yaml") as f:
    cfg = yaml.safe_load(f)

spark.sql(f"CREATE CATALOG IF NOT EXISTS {cfg['catalog_name']}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {cfg['catalog_name']}.{cfg['schema_name']}")

from src.extract.save_raw import run as extract_run
table_written = extract_run(cfg, spark)

df = spark.table(table_written)
display(df.limit(10))

In [0]:
spark.sql("DESCRIBE DETAIL main.weather_etl.raw_weather").display()

In [0]:
loc = spark.sql("DESCRIBE DETAIL main.weather_etl.raw_weather").select("location").collect()[0][0]
dbutils.fs.ls(loc)